In [ ]:
import math
import time
from google.colab import files
from itertools import chain

import gensim
from gensim.models import Word2Vec
import json

In [ ]:
# using value of each part of vector as distance, equally spacing around a point
# just complicated by presence of negative values, will probably bump everything up by minimum of whole dataset for now

def get_coordinates_for_vector(vector, min):
  points = []
  
  for c,v in enumerate(vector):
    point_angle = 2 * math.pi * c / len(vector)
    x = (v + min) * math.cos(point_angle)
    y = (v + min) * math.sin(point_angle)
    points.append([x, y])

  return points

In [ ]:
# getting positive version of lowest value in the model data, just flattening and getting the minimum with min, abs to make positive whether negative or positive
# all values will be increase by this so there are no negatives and the shape won't have points crossing over

def get_model_min(model):
  return abs(min(chain.from_iterable(model.wv['{}'.format(i)] for i in list(model.wv.vocab))))

In [ ]:
def generate_coords_all_words(model, min):
  all_word_coords = {}
  vocab = list(model.wv.vocab)
  for i in vocab:
    all_word_coords[i] = get_coordinates_for_vector(model.wv['{}'.format(i)], min)
  return all_word_coords

In [ ]:
def save_json(filename, data):
  with open(filename + '.json', 'w') as file_:
    json.dump(data, file_, ensure_ascii=False)
  time.sleep(200)
  files.download(filename + ".json")

In [ ]:
# at this point, will be about a quarter visible - the part that is both x and y positive, so need to pad
# for sake of comparisons best to do minimum for whole dataset

def make_svg_friendly(points, min_=None, viewbox=100):
  if not min_:
    min_x = min([i[0] for i in points])
    min_y = min([i[1] for i in points])

    padding = abs(min([min_x, min_y]) * 1.5)
  else:
    padding = abs(min_ * 1.5)
  # may later use viewbox size to try and scale the points to fit/fill better

  point_str = ""
  for point in points:
    point_str += ",".join([str(point[0] +padding), str(point[1] + padding)]) + " "

  return point_str.rstrip()

In [ ]:
def get_svg_min(coordinates_all_words): # coordinates_all_words will be dict with nested list values, want the smallest value from all, and can disregard keys
  all_x_coords = []
  all_y_coords = []

  for i in list(coordinates_all_words.values()):
    all_x_coords.append([j[0] for j in i])
    all_y_coords.append([j[1] for j in i])

  all_x_coords = list(chain.from_iterable(all_x_coords))
  all_y_coords = list(chain.from_iterable(all_y_coords))

  svg_min = abs(min([min(all_y_coords), min(all_x_coords)]))
  return svg_min


In [ ]:
def make_all_svg_polygon_strings(all_coords, offset): # all coords should be dict with word key and coordinate value as made above
  svg_dict = {}
  for k,v in all_coords.items():
    svg_dict[k] = make_svg_friendly(v, offset)
  return svg_dict

In [ ]:
# add path to model file to use below eg:
model = Word2Vec.load("scpword2vec.model")

# can do a quick check on the model with word expected to be in there eg:
# print(model.wv.most_similar('containment'))

In [ ]:
coordinates_all_words = generate_coords_all_words(model, get_model_min(model))

In [ ]:
polygon_str_dict = make_all_svg_polygon_strings(coordinates_all_words, get_svg_min(coordinates_all_words) )

In [ ]:
# give name for json file eg below, auto download set up to work if run in google colab
save_json( "scp_word_polygon_keyvals", polygon_str_dict)